In [114]:
from z3 import *
import numpy as np
from itertools import product
import time
import matplotlib.pyplot as plt

In [143]:
row_prods = np.array([], dtype=int)
col_prods = np.array([], dtype=int)

mb = lambda ch,N: np.array(IntVector(ch, N**2), dtype=object).reshape((N,N))
knight_moves = lambda i,j,N: [(i+di,j+dj) for di,dj in product(range(-2,3), repeat=2) if abs(di)+abs(dj)==3 and 0<=i+di<N and 0<=j+dj<N]
bishop_moves = lambda i,j,N: [(i+di,j+dj) for di,dj in product(range(-N,N), repeat=2) if abs(di)==abs(dj) and 0<=i+di<N and 0<=j+dj<N]
rook_moves = lambda i,j,N: [(i+di,j+dj) for di,dj in product(range(-N,N), repeat=2) if di*dj==0 and (di!=0 or dj!=0) and 0<=i+di<N and 0<=j+dj<N]
king_moves = lambda i,j,N: [(i+di,j+dj) for di,dj in product(range(-1,2), repeat=2) if di!=0 or dj!=0 and 0<=i+di<N and 0<=j+dj<N]
queen_moves = lambda i,j,N: bishop_moves(i,j,N) + rook_moves(i,j,N)

In [146]:
s = Solver()
N = len(row_prods)
queen, king, knight, bishop, rook, X = mb('q', N), mb('k', N), mb('n', N), mb('b', N), mb('r', N), mb('x', N)
boards = [(queen, queen_moves), (king, king_moves), (knight, knight_moves), (bishop, bishop_moves), (rook, rook_moves), (X, None)]

s += [And(n>=-1, n<=N) for board in boards for n in board[0].ravel()]
s += [If(board[0][i][j]>=0, X[i][j]==board[0][i][j], True) for (board,_) in boards[:-1] for (i,j),e in np.ndenumerate(board[0])]
s += [PbEq([(e==n, 1) for e in board[0].ravel()], 1) for n in range(1, N+1) for (board,_) in boards[:-1]]
s += [PbEq([(e==n, 1) for e in X.ravel()], len(boards)-1) for n in range(1, N+1)]

s += [Or(val<=0, PbEq([(board[k][l]==val-1, 1) for k,l in moves(i,j,N)], 1)) for (i,j),val in np.ndenumerate(X) for board,moves in boards[:-1]]
s += [Implies(board[i][j]==n, X[k][l]!=n) for (i,j),_ in np.ndenumerate(X) for k,l in moves(i,j,N) for n in range(1, N+1) for board,moves in boards[:-1]]

s += [np.prod(If(And(X[row][col]>0, X[row][col]<N), X[row][col], 1) for col in range(N))==row_prods[row] for row in range(N)]
s += [np.prod(If(And(X[row][col]>0, X[row][col]<N), X[row][col], 1) for row in range(N))==col_prods[col] for col in range(N)]




IndexError: index 0 is out of bounds for axis 0 with size 0